Reference text topic classification is good for supervised learning model or semi-supervised learning


1. perform Niave bayes classifer model to key words under each topics as baseline model

- tranform the class values to numerical variables as "encoded falgs"
- define X (text or sentences text) and y (classes of flags)
- Train: Whole text and their classes
- Test: tokenized sentences text and their classes
- then vetrioziing the dataset : CountVectorizer() on train then fit_transform the test
- then we can start to implment Naive Bayes model
- evaluate the model, get the most important features(words) by its coefficeint
- Tweek the model performance 

2. Try SVM/AGD CLassfier, Regularized Logistic Regression Model as better performing model

purpose:

- compare the performance
- comapre the important features 
- calaute the prediction confidence interval

can put them togather into a emsembed voting model?

use the reference_text classifer model as a perfect static model later to classify the transcript text ==> uased as probaiity score


3. generate refrence_text static bag of words, Static sub-Topics under refrence text Topic Extraction with Tf_IDF, LDA, and Vector Space Model  (pending)

How supervied learning model can influence unsupervised model?

the Important features(key words) from the supervised learning model can determin the subtopics extracted later with unsupervsied learning model 

- extract the sub-topics and key words from reference text under certin main topocs "refr_subtopics"
- then compare the extracted sub-topics from transcript text "transc_subtopics" to the "refr_subtopics" ==> get the scores such as using the frequency of the subtopics founds, compare the difference between the LDA coefficient or how key words formulate a subtopics, if the "transc_subtopics" has less or more subtopics learned, how those extra subtopics is related to the main topics?  


4. supervised learning model + unsupervised learning model can be used a semi-supervied learning model to produce 2 different set of the measurement to measure the relevance and factual scores

In [1]:
import sklearn
import pandas as pd
import numpy as np
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import wordcloud
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.ensemble import  VotingClassifier
import pickle

from Python_Functions.NLP_Data_Engineer_Functions import stopwords_generator, lemmatize_stemming, TF_whole_text, TF_IDF_text_doc, topic_extract, \
                          lemmatize_preprocess, text_cleaning_sub, text_cleaning, stop_words

In [2]:
df_train = pd.read_csv('ready_for_model_dataset/df_train.csv').drop('Unnamed: 0', axis = 1)

In [3]:
df = pd.read_csv("ready_for_model_dataset/ready_for_model.csv").drop('Unnamed: 0', axis = 1)

print(df.columns)

df.head()

Index(['Title', 'Video_ID', 'Category', 'Age_Restricted',
       'final_corrected_version_sentences_txt', 'final_corrected_version_txt',
       'sentence_level_timstamp_min_sec', 'sentence_level_timstamp_max_sec',
       'sentence_level_timstamp_min_minute',
       'sentence_level_timstamp_max_minute', 'duration', 'Length_(min)',
       'Views_(thous)', 'main_topics', 'reference_text',
       'reference_text_cleaned'],
      dtype='object')


,Title,Video_ID,Category,Age_Restricted,final_corrected_version_sentences_txt,final_corrected_version_txt,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,duration,Length_(min),Views_(thous),main_topics,reference_text,reference_text_cleaned
0,Children Learn About The Ant,cXUCUvcscXs,Ant,False,welcome to my tree house come and look,welcome to my tree house come and look. at thi...,0.560,4.080,0.009333,0.068000,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
1,Children Learn About The Ant,cXUCUvcscXs,Ant,False,at this,welcome to my tree house come and look. at thi...,2.560,6.080,0.042667,0.101333,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
2,Children Learn About The Ant,cXUCUvcscXs,Ant,False,this is my ant farm,welcome to my tree house come and look. at thi...,4.080,8.639,0.068000,0.143983,4.559,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
3,Children Learn About The Ant,cXUCUvcscXs,Ant,False,ants are amazing creatures lots of,welcome to my tree house come and look. at thi...,6.080,10.559,0.101333,0.175983,4.479,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
4,Children Learn About The Ant,cXUCUvcscXs,Ant,False,insects live by themselves and have to,welcome to my tree house come and look. at thi...,8.639,12.880,0.143983,0.214667,4.241,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...


### Mauanllay select and come up with Subtopics from the current reference txt

In [4]:

## from the PPT

birds_tailorbird_manual_keywords = ["tailorbird", 'colour', 'color' "appearances",\
                                    "beak", "food", 'voice', 'habitat', 'nest', "egg"  ]

birds_tailorbird_manual_useless_sentences = ['Hey I am Tinky  a Tailorbird. Do you know that I sing melodious songs? Let’s learn more about me!'] 


birds_peacock_manual_keywords = ["peacock", "peafowl", 'feature', 'specialty', 'train', 'crest', 'peahen',
                                 'green peafowl', 'white peafowl', 'colour', 'color' "appearances",\
                                    "beak", "food", 'voice', 'habitat', 'nest', "egg"  ]

birds_peacock_manual_useless_sentences = ['Source: peacocksuk.com', 'Source: pinterest.com', 'Source: natureconservation.in', \
                                         'Source: en.wikipedia.org'] 


birds_flamingo_manual_keywords = ["flamingo", 'feature', 'pink', 'specialty', 'train', 'carotenoids'
                                 'lesser flamingo', 'american flamingo',  'andean flamingo',\
                                 'chilean flamingo' , 'james flamingo', 'colour', 'color' "appearances",\
                                    "beak", "food", 'voice', 'habitat', 'nest', "egg", 'milk'  ]

birds_flamingo_manual_useless_sentences = ['Source: ebird.org', 'Source: en.wikipedia.org, commons.wikimedia.org'] 


insect_ant_manual_keywords = ["ant", 'queen', 'male', "drone", 'worker', 'social', 'princess', 'soldier', 'larva', 'larvae',\
                              'castes', 'black ants', 'appearance', 'habit', 'carpenter', 'weaver', 'leafcutter'\
                              "Odorous house", "food", 'nest', "egg", 'Life cycle'  ]

insect_ant_manual_useless_sentences = ['GLOSSARY: 1.Honeydew: a sweet sticky substance produced by insects while they feed on the plant sap. 2.Plant secretion: the produce of plants such as gums resins etc.'] 


insect_beetle_manual_keywords = ["beetle", 'elytra', 'wing', "ladybug", 'appearance', 'habit', 'scarab', 'mango stem borer', 'weevil', 'dung',\
                              'castes', 'black ants', 'appearance', 'habit', 'carpenter', 'weaver', 'leafcutter'\
                              "Odorous house", "food", 'nest', "egg", 'Life cycle'  ]

insect_beetle_manual_useless_sentences = ['We see many insects around us. Some have wings like mosquitoes, flies, butterflies, etc., while others do not have wings like ants, caterpillars, etc.',\
                                         '\\t', 'GLOSSARY: Species: A group of living organisms that are alike in their features and behaviour. Snout: a long nose. Millet: A type of cereal.' ] 

strings_to_remove = birds_tailorbird_manual_useless_sentences + birds_peacock_manual_useless_sentences + birds_flamingo_manual_useless_sentences +\
             insect_ant_manual_useless_sentences + insect_beetle_manual_useless_sentences

manual_reference_topics = {'bird_tailorbird' :  birds_tailorbird_manual_keywords,
                           'bird_peacock' : birds_peacock_manual_keywords, 
                           'bird_flamingo' : birds_flamingo_manual_keywords,
                          'insect_ant' : insect_ant_manual_keywords, 
                           'insect_beetle' : insect_beetle_manual_keywords}


#### start to extract the reference text and preproceossing refernece text column to be ready for label

In [5]:
import re

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', vTEXT, flags=re.MULTILINE)
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    vTEXT = vTEXT.replace("'/'","").replace("/","").replace("'https://'", '').replace("'[Music]'", '')
    return(vTEXT)

def form_txt_chunks(txt_lst, sep = ' '):
    
    if txt_lst != []:
        start_txt = ''
        for indx, sen in enumerate(txt_lst):
            if indx == 0:
                put_togather = start_txt + sen 
            else:
                put_togather = put_togather + sep + sen
        return put_togather
    else: return ""
        
def clean_txt_chunk(txt):
    
    sep_lst = txt.split(",")
    cleaned_lst = [i.replace("'", '').strip().replace("[", '').replace("]", '').replace(".jpg", '').replace(".html", '') for i in sep_lst]
    put_togather = form_txt_chunks(txt_lst = cleaned_lst, sep = ' ')

    return remove_urls (vTEXT = put_togather)



##### combine the train set (medium to excellent quanlity script) with the cleaned reference texts to help train the model 

- when combine classifier train set with the reference text ==> introduce more variance and more situation from medium to execellent quality video transcript to train the classifier, and help generlaize and predict the classified test set

In [6]:
refer_txt_df = df[['reference_text', 'main_topics']].drop_duplicates().reset_index(drop = True)

df_train_tobe_combined = df_train[['final_corrected_version_txt',\
                     'main_topics']].drop_duplicates().reset_index(drop = True).rename(columns = {'final_corrected_version_txt': 'reference_text' })

refer_txt_df = pd.concat([refer_txt_df, df_train_tobe_combined], axis = 0 )

refer_txt_df["reference_text_cleaned"] = refer_txt_df['reference_text'].apply(clean_txt_chunk)

# remove the useless sentecnes that could be affect the learning of the concepts/facts
refer_txt_df['reference_text_cleaned_2x'] = refer_txt_df['reference_text_cleaned']
for strg in strings_to_remove:
    refer_txt_df['reference_text_cleaned_2x'] = refer_txt_df['reference_text_cleaned_2x'].str.replace(strg, "")

# apply the manally selected sub-topic key words to the data frame
main_topics_lst = []
manual_topics_string = []
for key, manual_topics in manual_reference_topics.items():
    manual_topics_string.append(form_txt_chunks(txt_lst = manual_topics, sep = ','))
    main_topics_lst.append(key)


df_manual_topics = pd.DataFrame(list(dict(zip(main_topics_lst, manual_topics_string)).items()),columns = ['main_topics','manual_subtopics']) 
refer_txt_df = pd.merge(refer_txt_df, df_manual_topics, "left", 'main_topics')

refer_txt_df_new = refer_txt_df[[ 'reference_text_cleaned_2x', 'manual_subtopics', 'main_topics']]


# form the sentences level from the reference text

refertxt_sentences_lvl_df = pd.DataFrame()
for index, row in refer_txt_df_new.iterrows():
    refertxt_sentences_lvl_df_temp = pd.DataFrame()
    txt = row['reference_text_cleaned_2x']
    clss = row['main_topics']
    manual_subtopic = row['manual_subtopics']
    
    cleaned_temp = text_cleaning(txt = txt, stop_words = stop_words)
#     print(cleaned_temp[1])
#     print(cleaned_temp[2])
    cleaned_words_agg  = form_txt_chunks(txt_lst = cleaned_temp[1], sep = ',')
    vocabulary_agg  = form_txt_chunks(txt_lst =  cleaned_temp[2], sep = ',')# unique words
    tokens_sentences_lst = cleaned_temp[3]
#     tokens_words_sentences_lst = [form_txt_chunks(txt_lst = i, sep = ',') for i cleaned_temp[4]]
    
    refertxt_sentences_lvl_df_temp['sentences'] = tokens_sentences_lst
    refertxt_sentences_lvl_df_temp['unique_vocabulary_agg'] = vocabulary_agg
    refertxt_sentences_lvl_df_temp['cleaned_words_agg'] = cleaned_words_agg
    refertxt_sentences_lvl_df_temp['reference_text_cleaned']  = txt
    refertxt_sentences_lvl_df_temp['manual_subtopics']  = manual_subtopic
    refertxt_sentences_lvl_df_temp['main_topics'] = clss
    
    refertxt_sentences_lvl_df = pd.concat([refertxt_sentences_lvl_df, refertxt_sentences_lvl_df_temp], axis = 0)
    

whole_text_df =  refertxt_sentences_lvl_df[['unique_vocabulary_agg', 'cleaned_words_agg', 'reference_text_cleaned', 'manual_subtopics', 'main_topics']]
whole_text_df['sentences'] = whole_text_df['reference_text_cleaned']

whole_text_df = whole_text_df[['sentences', 'unique_vocabulary_agg', 'cleaned_words_agg', 'reference_text_cleaned', 'manual_subtopics', 'main_topics']].drop_duplicates()

refertxt_sentences_lvl_df_cleaned_classifier = pd.concat([refertxt_sentences_lvl_df, whole_text_df ], axis = 0).drop_duplicates()



/var/folders/7g/qq8zpbxn66x6z5czndqlhg640000gn/T/ipykernel_58429/1466293669.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  refer_txt_df['reference_text_cleaned_2x'] = refer_txt_df['reference_text_cleaned_2x'].str.replace(strg, "")


In [7]:
print(refertxt_sentences_lvl_df_cleaned_classifier.shape)
refertxt_sentences_lvl_df_cleaned_classifier.head()

(2219, 6)


,sentences,unique_vocabulary_agg,cleaned_words_agg,reference_text_cleaned,manual_subtopics,main_topics
0,ants are one of the most common insects that l...,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
1,we also see them in our homes,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
2,wherever we keep sweet foods we see lines of ants,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
3,we can quickly identify them by seeing their l...,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
4,the ants are of many sizes and colours,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant


##### combine the train set (medium to excellent quanlity script) with the cleaned reference texts to help train the model 

- when combine lda train set with the reference text ==> introduce more variance and more situation with variant sub-topics formation from good, excellent quality video transcript to train the LDA subtopic extractor model and NB classifier model


In [8]:
refertxt_sentences_lvl_df_cleaned_classifier.to_csv("ready_for_model_dataset/whole_reference_ready_for_train_model.csv")

### Train the Supervised Learning

##### Model preproceing, Train/test Split

In [9]:
# convert label to a binary numerical variable
refertxt_sentences_lvl_df_cleaned_classifier['main_topics_coded'] = refertxt_sentences_lvl_df_cleaned_classifier.main_topics.map({'bird_tailorbird' : 0, 'insect_ant': 1, 'bird_peacock' :2, 'insect_beetle' : 3,
       'bird_flamingo': 4})


In [10]:
# define X and y
X = refertxt_sentences_lvl_df_cleaned_classifier.sentences
y = refertxt_sentences_lvl_df_cleaned_classifier.main_topics_coded

print(f"X Shape: {X.shape}")
print(f"y Shape: {y.shape}")

X Shape: (2219,)
y Shape: (2219,)


In [11]:
vect_all = CountVectorizer()
# learn training data vocabulary, then use it to create a document-term matrix
vect_all.fit(X)
X_dtm = vect_all.transform(X)

## Building and Evaluating models

### classification with Naive bayes

classifier model will capture the likelihood each each sentences can be classified as certain main topics

1. this will tell us how likely a sentences between to a main topics but not under s subtopics
2. avg of the likelihood can be treated as how liklihood all the sentecnes combined to be calssified as a main topics; used as a threshold to evalaute the transcript sentences likelihood to a main class
3. the output liklihood when apply the model to the whole text ==> can be used a threshold to laster classify the liklihood of the whole video text to a main class


for now use all the refenrece text data as train set to build a quick a dirty classification model

- output the classifier model to pickle
- generate the top important features/works from classifier model

In [12]:
# initaie the Multiniomaol Niave Bayes Model
nb = MultinomialNB()
# train and time the model using X_train_dtm
%time nb.fit(X_dtm, y)

CPU times: user 1.08 ms, sys: 580 µs, total: 1.66 ms
Wall time: 1.46 ms


MultinomialNB()

### Add Hyperparameter tuning Grid Search to tune the model

In [13]:
from sklearn.model_selection import GridSearchCV

def Grid_Searching(model, param_gird, cv_num, scoring, X_data, y_data  ):
    Model_Grid = GridSearchCV(estimator = model, param_grid = param_gird, cv = cv_num, scoring='f1_micro',refit = True, n_jobs=-1, verbose = 5)
    Model_Grid.fit( X_data, y_data)
    print(Model_Grid.best_params_, Model_Grid.best_estimator_)
    return Model_Grid



In [14]:
np.logspace(0,-9, num=50)

array([1.00000000e+00, 6.55128557e-01, 4.29193426e-01, 2.81176870e-01,
       1.84206997e-01, 1.20679264e-01, 7.90604321e-02, 5.17947468e-02,
       3.39322177e-02, 2.22299648e-02, 1.45634848e-02, 9.54095476e-03,
       6.25055193e-03, 4.09491506e-03, 2.68269580e-03, 1.75751062e-03,
       1.15139540e-03, 7.54312006e-04, 4.94171336e-04, 3.23745754e-04,
       2.12095089e-04, 1.38949549e-04, 9.10298178e-05, 5.96362332e-05,
       3.90693994e-05, 2.55954792e-05, 1.67683294e-05, 1.09854114e-05,
       7.19685673e-06, 4.71486636e-06, 3.08884360e-06, 2.02358965e-06,
       1.32571137e-06, 8.68511374e-07, 5.68986603e-07, 3.72759372e-07,
       2.44205309e-07, 1.59985872e-07, 1.04811313e-07, 6.86648845e-08,
       4.49843267e-08, 2.94705170e-08, 1.93069773e-08, 1.26485522e-08,
       8.28642773e-09, 5.42867544e-09, 3.55648031e-09, 2.32995181e-09,
       1.52641797e-09, 1.00000000e-09])

In [15]:
# , False]
param_grid_gb = { 'alpha':np.logspace(0,-9, num=50)
              ,'fit_prior':[True, False]
                }

Best_model_MultinomialNBClassifier = Grid_Searching(model = nb, param_gird=param_grid_gb,\
                                        cv_num = 5, scoring = 'roc_auc', X_data = X_dtm, y_data = y  )



Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'alpha': 1.6768329368110066e-05, 'fit_prior': True} MultinomialNB(alpha=1.6768329368110066e-05)


In [16]:
Best_model_MultinomialNBClassifier

GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e+00, 6.55128557e-01, 4.29193426e-01, 2.81176870e-01,
       1.84206997e-01, 1.20679264e-01, 7.90604321e-02, 5.17947468e-02,
       3.39322177e-02, 2.22299648e-02, 1.45634848e-02, 9.54095476e-03,
       6.25055193e-03, 4.09491506e-03, 2.68269580e-03, 1.75751062e-03,
       1.15139540e-03, 7.54312006e-04...
       7.19685673e-06, 4.71486636e-06, 3.08884360e-06, 2.02358965e-06,
       1.32571137e-06, 8.68511374e-07, 5.68986603e-07, 3.72759372e-07,
       2.44205309e-07, 1.59985872e-07, 1.04811313e-07, 6.86648845e-08,
       4.49843267e-08, 2.94705170e-08, 1.93069773e-08, 1.26485522e-08,
       8.28642773e-09, 5.42867544e-09, 3.55648031e-09, 2.32995181e-09,
       1.52641797e-09, 1.00000000e-09]),
                         'fit_prior': [True, False]},
             scoring='f1_micro', verbose=5)

In [17]:
# feature importance
clf = Best_model_MultinomialNBClassifier
print(clf.best_estimator_)

# feature_names = vect_all.get_feature_names()
# coefs_with_fns = zip(feature_names, clf.best_estimator_.coef_[0])

# coefs_with_fns_df = pd.DataFrame(coefs_with_fns,
#                     columns=['feature', 'coefficient'])

# coefs_with_fns_df.sort_values(by='coefficient', ascending=False, inplace=True)
# ## keep only Noun words form the top importnat features/key words with nltk.pos_tag

# coefs_with_fns_df.head()

MultinomialNB(alpha=1.6768329368110066e-05)


In [18]:
# save the classifier model 

f = open('models/trained_models/NaiveBeyasClassifier_ReferenceText.pckl', 'wb')
pickle.dump([clf, vect_all], f)
f.close()


### Unsupervised Learning Model 

Implement the unsupervised learning techniques to extract subtopics, 

* How to dynamically filtering out the subtopic extracted useful to later comparison with the reference text sub-topic extraction? (research on any previous built-in NLP science library??)

#### Part 2: 

- Apply unsupervised topic extraction learning tecniques to extract sub-topics

- applied the LAD extracted sub-topics model from different technqiues combine aginats the mnaual extraction to form a static subtopics aggreation and their related words that amounts to the subtopics 

  * this process will requires some manual selection and mnaual labeling on the subtopics extracted

- Expore and optimize the extracted topical results 


In [19]:
import time
import math
import re
from textblob import TextBlob
import pandas as pd

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# 
# import summa 

import string
import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

In [20]:
def summarize_text(text, ratio = 0.85):
    textSum = summa.summarizer.summarize(text = text, ratio=ratio)
    return textSum

def summarize_keywords_from_text(text):
    keyword_lst = summa.keywords.keywords(text)
    keyword_str = form_txt_chunks( txt_lst = keyword_lst, sep = ',')
    return keyword_str

def form_txt_chunks(txt_lst, sep = ' '):

    start_txt = ''
    for indx, sen in enumerate(txt_lst):
        if indx == 0:
            put_togather = start_txt + sen 
        else:
            put_togather = put_togather + sep + sen
    return put_togather

In [21]:
df = pd.read_csv("ready_for_model_dataset/whole_reference_ready_for_train_model.csv")
df.head()

,Unnamed: 0,sentences,unique_vocabulary_agg,cleaned_words_agg,reference_text_cleaned,manual_subtopics,main_topics
0,0,ants are one of the most common insects that l...,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
1,1,we also see them in our homes,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
2,2,wherever we keep sweet foods we see lines of ants,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
3,3,we can quickly identify them by seeing their l...,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant
4,4,the ants are of many sizes and colours,"castes,called,together,secretion,habitat,almos...","ants,one,common,insects,live,garden,also,see,h...",Ants are one of the most common insects that l...,"ant,queen,male,drone,worker,social,princess,so...",insect_ant


In [22]:
df_ref_txt = df[['reference_text_cleaned', 'manual_subtopics', 'main_topics' ]].drop_duplicates()
manaul_subtopics = list(df['manual_subtopics'].drop_duplicates())

sentences_lvl = {}
for main_topic in list(df['main_topics'].unique()):
    sentences_lvl[main_topic] = list(df[df['main_topics'] == main_topic]['sentences'])

In [23]:
import multiprocessing

num_processors = multiprocessing.cpu_count()
num_processors

def LDA_Topic_Extraction(doc_lst, topic_n, word_num, pass_n = 50 ):
    
    def clean(doc):
        stop_free = " ".join([i for i in doc.lower().replace("_", " ").split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized

    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    doc_clean = [clean(doc).split() for doc in doc_lst] 
    
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. 
    dictionary = corpora.Dictionary(doc_clean)

    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    
    # apply LAD to extract topics using gensim library
    Lda = gensim.models.ldamodel.LdaModel
    
    ldamodel = LdaMulticore(doc_term_matrix, 
                              num_topics=topic_n, 
                              id2word = dictionary, 
                              passes=pass_n, 
                              workers = num_processors-1) 
    print(*ldamodel.print_topics(num_topics=topic_n, num_words=word_num), sep='\n')
    
    return ldamodel, dictionary, doc_term_matrix

In [24]:
tailorbird_txt_doc_complete = [list(df_ref_txt['reference_text_cleaned'])[0]]
ant_txt_doc_complete = [list(df_ref_txt['reference_text_cleaned'])[1]]
peacock_txt_doc_complete = [list(df_ref_txt['reference_text_cleaned'])[2]]
beetle_txt_doc_complete = [list(df_ref_txt['reference_text_cleaned'])[3]]
flamingo_txt_doc_complete = [list(df_ref_txt['reference_text_cleaned'])[4]]

# apply topic extraction on reference txts
main_names = list(df['main_topics'].unique())
main_topics_sentences_lst = [tailorbird_txt_doc_complete, ant_txt_doc_complete, peacock_txt_doc_complete,\
                            beetle_txt_doc_complete, flamingo_txt_doc_complete]
main_topics_sentences_lst_dict = dict(zip(main_names, main_topics_sentences_lst))

for key, val in main_topics_sentences_lst_dict.items():
    main_topics_sentences_lst_dict[key] = val + sentences_lvl[key]

#### Reference Txt and Sentences Topic Extraction

In [25]:
# Tailor bird reference Topic Extraction
topics_list = list(df['main_topics'].unique())
lda_model = []
int_idx = 0
for main_topics in topics_list :
    print("----------------------------------")
    print("{} sub-topics Extraction: ".format(main_topics))
    print(manaul_subtopics[int_idx])
    manual_topic_n = len(manaul_subtopics[int_idx].split(","))
    print("Tailor Bird Has {} manaul sub-topics".format(len(manaul_subtopics[int_idx].split(","))))

    # extract 
    extracted_topics_lda_model = LDA_Topic_Extraction(doc_lst = main_topics_sentences_lst_dict[main_topics], topic_n = manual_topic_n * 2, word_num = 10)
    temp_dict = {}
    temp_dict[main_topics] = extracted_topics_lda_model
    lda_model.append(temp_dict)
    print("----------------------------------")
    int_idx += 1


----------------------------------
insect_ant sub-topics Extraction: 
ant,queen,male,drone,worker,social,princess,soldier,larva,larvae,castes,black ants,appearance,habit,carpenter,weaver,leafcutterOdorous house,food,nest,egg,Life cycle
Tailor Bird Has 21 manaul sub-topics
(0, '0.065*"sting" + 0.055*"ant" + 0.044*"city" + 0.028*"worker" + 0.026*"theyre" + 0.024*"highly" + 0.024*"industrious" + 0.024*"exact" + 0.024*"handy" + 0.022*"powerful"')
(1, '0.130*"ant" + 0.029*"also" + 0.024*"house" + 0.024*"nest" + 0.022*"food" + 0.020*"live" + 0.020*"build" + 0.019*"black" + 0.015*"leaf" + 0.015*"carpenter"')
(2, '0.029*"tend" + 0.025*"else" + 0.024*"well" + 0.022*"leaf" + 0.020*"ant" + 0.019*"queen" + 0.018*"baby" + 0.015*"worker" + 0.015*"instead" + 0.014*"assigned"')
(3, '0.055*"trail" + 0.051*"new" + 0.045*"ant" + 0.036*"one" + 0.033*"leave" + 0.028*"behind" + 0.028*"growing" + 0.027*"home" + 0.023*"impossible" + 0.023*"neat"')
(4, '0.050*"like" + 0.043*"system" + 0.043*"make" + 0.027*"nes

(0, '0.058*"america" + 0.058*"south" + 0.029*"flamingo" + 0.029*"dont" + 0.029*"know" + 0.029*"say" + 0.029*"spanish" + 0.029*"come" + 0.029*"gu" + 0.029*"uh"')
(1, '0.065*"flamingo" + 0.020*"one" + 0.020*"bird" + 0.017*"pink" + 0.017*"egg" + 0.011*"mud" + 0.011*"however" + 0.011*"foot" + 0.011*"know" + 0.011*"chick"')
(2, '0.040*"like" + 0.040*"see" + 0.040*"flamingo" + 0.039*"milk" + 0.039*"talk" + 0.021*"gonna" + 0.021*"produce" + 0.020*"look" + 0.020*"well" + 0.020*"video"')
(3, '0.052*"thats" + 0.049*"egg" + 0.046*"later" + 0.044*"sure" + 0.038*"right" + 0.035*"know" + 0.034*"year" + 0.031*"make" + 0.028*"one" + 0.027*"also"')
(4, '0.045*"every" + 0.045*"slightly" + 0.045*"larger" + 0.045*"entire" + 0.040*"voice" + 0.035*"female" + 0.033*"depending" + 0.029*"dont" + 0.027*"male" + 0.026*"year"')
(5, '0.076*"flamingo" + 0.055*"subscribe" + 0.049*"adult" + 0.042*"theyre" + 0.040*"one" + 0.038*"depending" + 0.030*"goodness" + 0.030*"longer" + 0.030*"find" + 0.030*"flushed"')
(6, '0.0

(0, '0.089*"tailorbird" + 0.064*"egg" + 0.028*"nest" + 0.022*"beak" + 0.019*"lay" + 0.018*"food" + 0.017*"plant" + 0.017*"needle" + 0.016*"fibre" + 0.014*"like"')
(1, '0.110*"bird" + 0.095*"nest" + 0.045*"weaver" + 0.041*"build" + 0.027*"open" + 0.027*"also" + 0.022*"small" + 0.021*"swallow" + 0.018*"barn" + 0.018*"interior"')
(2, '0.162*"bird" + 0.115*"female" + 0.076*"male" + 0.047*"feed" + 0.037*"parent" + 0.015*"protect" + 0.014*"cattail" + 0.014*"like" + 0.013*"build" + 0.012*"inside"')
(3, '0.044*"web" + 0.033*"sun" + 0.033*"silk" + 0.032*"spider" + 0.023*"nature" + 0.021*"fiber" + 0.020*"make" + 0.020*"lot" + 0.020*"nest" + 0.017*"outside"')
(4, '0.065*"bird" + 0.049*"like" + 0.028*"egg" + 0.013*"look" + 0.010*"the" + 0.009*"next" + 0.009*"are" + 0.009*"small" + 0.009*"chick" + 0.009*"fly"')
(5, '0.028*"garden" + 0.028*"paper" + 0.027*"bird" + 0.021*"might" + 0.020*"live" + 0.019*"entrance" + 0.016*"rufus" + 0.014*"hand" + 0.014*"already" + 0.014*"narrow"')
(6, '0.037*"nest" + 0

In [26]:
topics_temp = lda_model[0]['insect_ant'][0].print_topics()

### Visualize the Extracted Sub-topics 

In [27]:
# pip install pyLDAvis==2.1.2

In [28]:
import pyLDAvis
#import pyLDAvis.gensim
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

mdl = lda_model[0]['insect_ant'][0]
id2word  = lda_model[0]['insect_ant'][1]
corpus  = lda_model[0]['insect_ant'][2]

vis = pyLDAvis.gensim_models.prepare(mdl, corpus, id2word,  mds='mmds')
pyLDAvis.display(vis)  

/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [29]:
# save the trained subtopic extrcation model
import pickle

f = open('models/trained_models/LDA_Subtopic_Models_ReferenceText.pckl', 'wb')
pickle.dump(lda_model, f)
f.close()

/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


[CV 4/5] END .........alpha=1.0, fit_prior=True;, score=0.793 total time=   0.0s
[CV 1/5] END alpha=0.6551285568595508, fit_prior=True;, score=0.851 total time=   0.0s
[CV 5/5] END alpha=0.6551285568595508, fit_prior=True;, score=0.679 total time=   0.0s
[CV 4/5] END alpha=0.6551285568595508, fit_prior=False;, score=0.795 total time=   0.0s
[CV 4/5] END alpha=0.4291934260128778, fit_prior=True;, score=0.815 total time=   0.0s
[CV 3/5] END alpha=0.4291934260128778, fit_prior=False;, score=0.741 total time=   0.0s
[CV 2/5] END alpha=0.1842069969326716, fit_prior=True;, score=0.815 total time=   0.0s
[CV 2/5] END alpha=0.12067926406393285, fit_prior=False;, score=0.784 total time=   0.0s
[CV 3/5] END alpha=0.12067926406393285, fit_prior=False;, score=0.806 total time=   0.0s
[CV 4/5] END alpha=0.07906043210907697, fit_prior=False;, score=0.851 total time=   0.0s
[CV 5/5] END alpha=0.07906043210907697, fit_prior=False;, score=0.612 total time=   0.0s
[CV 4/5] END alpha=0.0517947467923121, 

[CV 3/5] END ........alpha=1.0, fit_prior=False;, score=0.712 total time=   0.0s
[CV 4/5] END alpha=0.6551285568595508, fit_prior=True;, score=0.809 total time=   0.0s
[CV 5/5] END alpha=0.4291934260128778, fit_prior=True;, score=0.688 total time=   0.0s
[CV 4/5] END alpha=0.4291934260128778, fit_prior=False;, score=0.806 total time=   0.0s
[CV 1/5] END alpha=0.07906043210907697, fit_prior=True;, score=0.892 total time=   0.0s
[CV 2/5] END alpha=0.07906043210907697, fit_prior=True;, score=0.831 total time=   0.0s
[CV 3/5] END alpha=0.0517947467923121, fit_prior=True;, score=0.842 total time=   0.0s
[CV 4/5] END alpha=0.0517947467923121, fit_prior=True;, score=0.863 total time=   0.0s
[CV 1/5] END alpha=0.03393221771895328, fit_prior=True;, score=0.905 total time=   0.0s
[CV 2/5] END alpha=0.03393221771895328, fit_prior=True;, score=0.842 total time=   0.0s
[CV 3/5] END alpha=0.03393221771895328, fit_prior=True;, score=0.847 total time=   0.0s
[CV 4/5] END alpha=0.03393221771895328, fit

/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


[CV 1/5] END .........alpha=1.0, fit_prior=True;, score=0.840 total time=   0.0s
[CV 2/5] END alpha=0.6551285568595508, fit_prior=True;, score=0.784 total time=   0.0s
[CV 1/5] END alpha=0.6551285568595508, fit_prior=False;, score=0.813 total time=   0.0s
[CV 5/5] END alpha=0.6551285568595508, fit_prior=False;, score=0.650 total time=   0.0s
[CV 5/5] END alpha=0.281176869797423, fit_prior=True;, score=0.679 total time=   0.0s
[CV 1/5] END alpha=0.1842069969326716, fit_prior=True;, score=0.874 total time=   0.0s
[CV 5/5] END alpha=0.1842069969326716, fit_prior=True;, score=0.668 total time=   0.0s
[CV 1/5] END alpha=0.12067926406393285, fit_prior=True;, score=0.878 total time=   0.0s
[CV 2/5] END alpha=0.12067926406393285, fit_prior=True;, score=0.820 total time=   0.0s
[CV 5/5] END alpha=0.07906043210907697, fit_prior=True;, score=0.657 total time=   0.0s
[CV 1/5] END alpha=0.07906043210907697, fit_prior=False;, score=0.863 total time=   0.0s
[CV 1/5] END alpha=0.00021209508879201905, 

/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
